All data downloaded from https://www.football-data.co.uk/spainm.php

In [29]:
import numpy as np
import pandas as pd
import re

In [30]:
league_names = ['primera', 'segunda']
league_years = [f"{year}_{year+1}" for year in range(1999, 2023)]

match_result_cols = [
    'Div', # League Division
    'Date', # Match Date (dd/mm/yy)
    'Time', # Time of match kick off
    'HomeTeam', # Home Team
    'AwayTeam', # Away Team
    'FTHG and HG', # Full Time Home Team Goals
    'FTAG and AG', # Full Time Away Team Goals
    'FTR and Res', # Full Time Result (H=Home Win, D=Draw, A=Away Win)
    'HTHG', # Half Time Home Team Goals
    'HTAG', # Half Time Away Team Goals
    'HTR', # Half Time Result (H=Home Win, D=Draw, A=Away Win)
]

match_stats_cols = [
    'Attendance', # Crowd Attendance
    'Referee', # Match Referee
    'HS', # Home Team Shots
    'AS', # Away Team Shots
    'HST', # Home Team Shots on Target
    'AST', # Away Team Shots on Target
    'HHW', # Home Team Hit Woodwork
    'AHW', # Away Team Hit Woodwork
    'HC', # Home Team Corners
    'AC', # Away Team Corners
    'HF', # Home Team Fouls Committed
    'AF', # Away Team Fouls Committed
    'HFKC', # Home Team Free Kicks Conceded
    'AFKC', # Away Team Free Kicks Conceded
    'HO', # Home Team Offsides
    'AO', # Away Team Offsides
    'HY', # Home Team Yellow Cards
    'AY', # Away Team Yellow Cards
    'HR', # Home Team Red Cards
    'AR', # Away Team Red Cards
    'HBP', # Home Team Bookings Points (10', # yellow, 25', # red)
    'ABP', # Away Team Bookings Points (10', # yellow, 25', # red)
]

Reading all the data from files into separate dataframes; which will later be consolidated into a single one.

In [31]:
# This dictionary will hold every single separate dataframe
df_list = dict()

# Going through all the files for each league division
for league_year in league_years:
    for league_name in league_names:
        cur_league = f"{league_name}_{league_year}"
        # Reading the league's data from the file
        cur_df = pd.read_csv("./initial_ds/" + cur_league + ".csv")
        # Removing the columns corresponding to betting odds
        cur_df = cur_df[[col for col in cur_df if col in match_result_cols+match_stats_cols]]
        # Removing the rows which contain only Null values
        cur_df = cur_df.dropna(axis='index', how="all")
        # Removing the columns which contain only Null values
        cur_df = cur_df.dropna(axis="columns", how='all')
        df_list[cur_league] = cur_df
        
print(f"{len(df_list)} dataframes were read.")

df_list.keys()

48 dataframes were read.


dict_keys(['primera_1999_2000', 'segunda_1999_2000', 'primera_2000_2001', 'segunda_2000_2001', 'primera_2001_2002', 'segunda_2001_2002', 'primera_2002_2003', 'segunda_2002_2003', 'primera_2003_2004', 'segunda_2003_2004', 'primera_2004_2005', 'segunda_2004_2005', 'primera_2005_2006', 'segunda_2005_2006', 'primera_2006_2007', 'segunda_2006_2007', 'primera_2007_2008', 'segunda_2007_2008', 'primera_2008_2009', 'segunda_2008_2009', 'primera_2009_2010', 'segunda_2009_2010', 'primera_2010_2011', 'segunda_2010_2011', 'primera_2011_2012', 'segunda_2011_2012', 'primera_2012_2013', 'segunda_2012_2013', 'primera_2013_2014', 'segunda_2013_2014', 'primera_2014_2015', 'segunda_2014_2015', 'primera_2015_2016', 'segunda_2015_2016', 'primera_2016_2017', 'segunda_2016_2017', 'primera_2017_2018', 'segunda_2017_2018', 'primera_2018_2019', 'segunda_2018_2019', 'primera_2019_2020', 'segunda_2019_2020', 'primera_2020_2021', 'segunda_2020_2021', 'primera_2021_2022', 'segunda_2021_2022', 'primera_2022_2023', 's

All columns available in one year are also available in the next:

In [32]:
# Comparing the dataframe for each season with the next season and printing the list of the columns that
# are available in the older season and unavailable in the newer
for league_name in league_names:
    for i in range(len(league_years) - 1):
        cur_league = f"{league_name}_{league_years[i]}"
        next_league = f"{league_name}_{league_years[i+1]}"
        unmatched_cols = [col for col in df_list[cur_league].columns if col not in df_list[next_league].columns]
        if unmatched_cols:
            print(f"columns in {cur_league} and unavailable in {next_league}:\n{unmatched_cols}\n")

So the dataframes can be merged into a single one with the columns corresponding to the most recent one:

In [33]:
# Merging all the dataframes of different seasons of the same league together
for league_name in league_names:
    df_list[league_name] = pd.concat(
        [df_list[league] for league in df_list.keys() if league_name in league],
        axis='index'
    )
    print(league_name, "\n", df_list[league_name].columns)  

primera 
 Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'HTHG', 'HTAG', 'HTR', 'HS',
       'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR',
       'Time'],
      dtype='object')
segunda 
 Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'HTHG', 'HTAG', 'HTR', 'HS',
       'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR',
       'Time'],
      dtype='object')


Since the columns in both merged dataframes are the same, they can be merged together too.

In [34]:
# Changing the values in the 'Div' (Division) column from SP1 and SP2 to the actual league name
for league_name in league_names:
    df_list[league_name]['Div'] = league_name
# Merging the dataframes corresponding to each league together
df = pd.concat([df_list[league_name] for league_name in league_names],
               axis='index')

In [35]:
df.to_csv("./initial_ds/matches.csv", index=False)